# blah blah

In [1]:

import numpy as np
import pandas as pd

#######################
## Datenaufbereitung ##
#######################


# make imports from pa_lib possible (parent directory of file's directory)
import sys
from pathlib import Path
                                
file_dir = Path.cwd()
parent_dir = file_dir.parent
sys.path.append(str(parent_dir))


from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows  = None

## Libraries & Settings ##
from pa_lib.file import load_bin
from pa_lib.util import cap_words
from pa_lib.log import time_log, info

import datetime as dt
from dateutil.relativedelta import relativedelta

from pa_lib.data import (
    clean_up_categoricals,
    unfactorize,
)

from pa_lib.data import desc_col

In [4]:
def load_booking_data():
    bd_raw = load_bin("vkprog\\bd_data.feather").rename(
        mapper=lambda name: cap_words(name, sep="_"), axis="columns"
    )
    bd = bd_raw.loc[(bd_raw.Netto > 0)].pipe(clean_up_categoricals)
    return bd

In [5]:
bd_raw = load_booking_data()

2019-10-29 16:44:45 [INFO] Started loading binary file
2019-10-29 16:44:45 [INFO] Reading from file C:\Users\stc\data\vkprog\bd_data.feather
2019-10-29 16:44:45 [INFO] Finished loading binary file in 0.15s (0.77s CPU)


In [2]:
ek_info = load_bin("vkprog\\ek_info.feather")

2019-10-29 16:44:08 [INFO] Started loading binary file
2019-10-29 16:44:08 [INFO] Reading from file C:\Users\stc\data\vkprog\ek_info.feather
2019-10-29 16:44:09 [INFO] Finished loading binary file in 0.06s (0.06s CPU)


In [3]:
desc_col(ek_info)

,DTYPE,NULLS,UNIQUE
index,int64,0/32237,32237
Endkunde_NR,int64,0/32237,32237
Endkunde,object,0/32237,30291
EK_Aktiv,int64,0/32237,2
EK_Kam_Betreut,int64,0/32237,2
EK_Land,object,49/32188,45
PLZ,int64,0/32237,2875
GEMEINDE,object,49/32188,3006
Agentur,object,20999/11238,3247
EK_BG,object,1829/30408,83


# OMG! I FORGOT ABOUT THE IMPLICIT USE OF COLUMNS IN THE ROW SELECTION!!!

- ``Kleinkunde`` (Über die letzten 4 Jahre nie mehr als 3'000 Umsatz pro Jahr)
- ``Neukunde`` (Alle, die erst im aktuellen Jahr Umsatz hatten)
- ``Insolvenz`` (Kunden mit Bonitäts-Flag "keine Verkäufe" werden markiert)
- ``Umsatz_erreicht`` (80% Netto-Umsatz gem. Vorjahr erreicht)
- ``kuerzlich_gebucht`` (in den letzten 2 Monaten erfasste Kampagnen )
- ``kuerzlich_im_aushang`` (Aushangbeginn vor 1 Monat oder später)
- ``kuerzlich_im_kontakt`` (CRM-Kontakt in den letzten 4 Wochen)
- ``VB_FILTER_AKTIV`` (in CRM ist eine gültige Sperre für Kunden erfasst) 

In [6]:
bd_raw.sample(5, random_state=42)

,Endkunde_NR,Endkunde,EK_Abc,EK_Boni,EK_Plz,EK_Ort,EK_Land,EK_HB_Apg_Kurzz,EK_Kam_Betreut,EK_Aktiv,Agentur,AG_Hauptbetreuer,Verkaufsberater,Endkunde_Branchengruppe_ID,Endkunde_Branchengruppe,Endkunde_Nbranchengruppe_ID,Endkunde_Nbranchengruppe,Endkunde_Branchenkat_ID,Endkunde_Branchenkat,Endkunde_Nbranchenkat_ID,Endkunde_Nbranchenkat,Auftrag_Branchengruppe_ID,Auftrag_Branchengruppe,Auftrag_Nbranchengruppe_ID,Auftrag_Nbranchengruppe,Auftrag_Branchenkat_ID,Auftrag_Branchenkat,Auftrag_Nbranchenkat_ID,Auftrag_Nbranchenkat,Agps_NR,Segment,KV_NR,KV_Typ,Kampagnen_Status,Kampagne_Erfassungsdatum,Kampagne_Beginn,Auftragsart,Res_Dat,Annullation_Datum,Aush_Von,Dauer,Vertrag,Brutto,Netto,Agglo,PF,Kamp_Beginn_Jahr,Kamp_Beginn_KW,Kamp_Beginn_KW_2,Kamp_Beginn_KW_4,Kamp_Erfass_Jahr,Kamp_Erfass_KW,Kamp_Erfass_KW_2,Kamp_Erfass_KW_4
1017991,592834,Carta turistica SA,E,gut,6901,Lugano,SCHWEIZ,SOL,0,1,NaN,NaN,ROS,405,WG - Reisen / Hotels / Sanatorien,NaN,NaN,10,WB - Freizeit / Touristik,NaN,NaN,405,WG - Reisen / Hotels / Sanatorien,NaN,NaN,10,WB - Freizeit / Touristik,NaN,NaN,2050838,APG|SGA,811509,KPG,4,2015-03-10,2015-06-15,Kommerziell,2015-03-27,NaT,2015-06-15,14,Nein,3733,3733,"A5192,A5250",F12,2015,25,25,25,2015,11,11,9
1197937,168605,C&A Mode AG,A,gut,6341,Baar,SCHWEIZ,DEA,1,1,"R. Koller & L. Della Piana Koller, Consulting",KUR,KUR,195,WG - Bekleidung / Wäsche,901,WG - Handel / Grossverteiler,04,WB - Bekleidung / persönlicher Bedarf,14,WB - Handel,195,WG - Bekleidung / Wäsche,901,WG - Handel / Grossverteiler,04,WB - Bekleidung / persönlicher Bedarf,14,WB - Handel,1813376,APG|SGA,754283,KPG,4,2013-11-26,2014-05-05,Kommerziell,2013-11-26,NaT,2014-05-05,14,Nein,2428,1991,"90306,90329,90404,91103,91509,92583,93293,9378...","City eBoard,City ePanel,Escalator ePanel,F12,F...",2014,19,19,17,2013,48,47,45
1137867,536344,Küry Automobile AG,D,gut,8803,Rüschlikon,SCHWEIZ,MOE,0,1,X-TRA RAINER TRUTTMANN,BGI,DSC,400,WG - Automarkt,NaN,NaN,13,WB - Verkehr,NaN,NaN,400,WG - Automarkt,NaN,NaN,13,WB - Verkehr,NaN,NaN,1768257,Traffic,742309,KDVTLG,4,2013-09-01,2013-12-01,Traffic Auftrag,2013-09-01,NaT,2013-12-01,365,Ja,7500,7500,A0261,"F12,F24,Ganzheck",2013,48,47,45,2013,35,35,33
865735,104912,APG|SGA Rail,A,gut,8027,Zürich,SCHWEIZ,YAM,0,1,NaN,NaN,JZE,720,WG - Dienstleistung,NaN,NaN,15,WB - Dienstleistung,NaN,NaN,980,WG - Spitäler / Kliniken / Kurzentren / Aerzte,NaN,NaN,21,WB - Spitäler/ Kliniken/ Kurzentren/ Aerzte,NaN,NaN,1672038,Rail,717049,KPG,4,2013-01-09,2013-01-28,Aushangauftrag Partner,2013-01-09,NaT,2013-01-28,7,Nein,576,576,"90306,90329,90404,90412,90768,91103,91362,9137...","F12,F12L,F200,F200L,F24,F4,F4C,F4S,NeonSign,Pr...",2013,5,5,5,2013,2,1,1
165668,144291,AllBlues Konzert AG,D,gut,8404,Winterthur,SCHWEIZ,ACO,0,1,NaN,NaN,ACO,990,WG - Veranstaltungen,NaN,NaN,22,WB - Veranstaltungen,NaN,NaN,990,WG - Veranstaltungen,NaN,NaN,22,WB - Veranstaltungen,NaN,NaN,2798438,Digital Sales,968021,KPG,4,2018-10-04,2018-10-15,Kommerziell,2018-10-04,NaT,2018-10-29,1,Nein,168,168,"A0230,A0261,A0351,A1061,A2701,A3203,A4001,A558...","City eBoard,City ePanel,Escalator ePanel,F12,F...",2018,42,41,41,2018,40,39,37


In [8]:
set(bd_raw.loc[:,"EK_Boni"])

{'gegen Vorauszahlung', 'gut', 'keine Verkäufe', nan}